In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os,IPython, librosa, mir_eval
from os import listdir
from os.path import isfile, join,isdir
from IPython.display import Audio
from librosa.display import waveplot,specshow
from librosa.onset import onset_strength, onset_detect
from librosa.feature import melspectrogram, mfcc

from collections import defaultdict,OrderedDict
import sklearn
from sklearn.preprocessing import StandardScaler,LabelEncoder
import scipy
from pandas import HDFStore,DataFrame


# Song Extraction from fma_small

1. Find the song folder path relative to the current computer
2. Retrieve the different genre classifications
3. Identify each song via its full path to song using index, to guarantee one-to-one mapping 
4. Sort in alphabetical order

In [ ]:
#retrieving path to the fma_small directory and the corresponding meta data
HOME_DIR = IPython.utils.path.get_home_dir()

temp = join(HOME_DIR, 'Documents')
path_to_small_fma = join(temp, 'fma_small')
json_file = join(path_to_small_fma,'fma_small.json')
#locate meta_dta
print(path_to_small_fma)
df = pd.read_json(json_file)
print(json_file)


In [ ]:
#only choose top genre as the label
df = df.loc[:,['top_genre']]

#ensure that the genre name matches file name in fma_small, 
#i.e Oldtime / Historian conflict issue
df['top_genre']=df['top_genre'].apply(lambda y: y.split(os.sep)[0].strip())
#locate each individual song by its full path
df['temp'] = path_to_small_fma
str_index = ["%.2d" % x for x in df.index]
complete_genre_list = df['top_genre'].unique()
le = LabelEncoder()
encoded_genres= le.fit(complete_genre_list)
print(complete_genre_list)

#create full path to file and store as a single array
df['full_path_to_song'] = df.temp.map(str)+ "/"+ df['top_genre'].values+ "/"+ str_index+ ".mp3"
del df['temp']
#keep songs according to alphabetical order of songs 
df.sort_values(by = 'top_genre', inplace = True)
df.head()


In [ ]:
#retrieve number of songs per genre
genre_and_count = df['top_genre'].value_counts().sort_index()
all_songs_path = df['full_path_to_song'].values  
print(genre_and_count)

In [ ]:
song_counts = [] 
ordered_genres = [] 
full_song_df = OrderedDict()
genre_to_song_dict = {}


#retrieve number of songs per genre with the order preserved in two lists
#ordered_genres
#song_counts
for i,genre in enumerate(genre_and_count.index):
    ordered_genres.append(genre)
    temp = df['top_genre'].value_counts()[genre]
    song_counts.append(temp)

genre_to_song_zipped = zip(ordered_genres,song_counts)

num_of_genre = np.shape(genre_and_count)[0]
print("The number of genres is:", num_of_genre)

#dictionary with each song count and its corresponding genre
for genre,song_count in genre_to_song_zipped:
    genre_to_song_dict[genre] = song_count

genre_to_song_dict

In [ ]:
num_of_genre = np.shape(genre_and_count)[0]
all_songs_path = df['full_path_to_song'].values

paths_dict = OrderedDict()

#prepend zero so we have a start point for all_songs_path 
#and avoid messing with indices

#use the cumulative sum to find none uniform ranges
song_counts.insert(0,0)
cumulative_sum = np.cumsum(song_counts,dtype=int)

#creates a dictionary of the genres and its corresponding path
for i,genre in enumerate(ordered_genres):
    str1=genre
    str2 = "_paths"
    genre_paths = "".join((str1,str2))
    paths_dict[genre_paths] = all_songs_path[cumulative_sum[i]:cumulative_sum[i+1]]

#paths_dict
#{genre_path_name: genre_paths}
print("{'Electronic_paths:[array_of_all_electronic_paths]}")

##### assumed here that songs are Mono with sampling rate 22050

In [ ]:
%%time
num_of_songs = 3
sampling_rate = 44100

genre_signals_dict = OrderedDict()
#creates a dictionary of the signals in a genre and their raw file
for genre_path_name,genre_paths in paths_dict.items():
    str1=genre_path_name[:-5]
    str2 = "signals"
    genre_signals = "".join((str1,str2))       
    try:
        first_three = genre_paths[:num_of_songs]
        genre_signals_dict[genre_signals] = [
        librosa.load(p,sr=sampling_rate,mono=True)[0] for p in first_three]
    except IOError as exc:
        print("Unable to locate folder")
        #raise IOError("%s: %s" % (genre_paths, exc.strerror))
        
#genre_signals_dict
#{genre_signals_name:genre_signals_paths}
print("{'Electronic_signals:[array_of_all_electronic_paths]}")

In [ ]:
#plot the time series for each song according to the genres

#sig_lengths = []
for genre_signal_name,genre_signals in genre_signals_dict.items(): 
    for i, sig_amp in enumerate(genre_signals):
        plt.subplot(1, num_of_songs, i+1)
#        sig_lengths.append(len(sig_amp))
        librosa.display.waveplot(sig_amp)
        plt.ylim(-1, 1)
        plt.title(genre_signal_name)
    plt.figure()
        
    

# Basic Work Flow

## Segmentation - > Feature Extraction -> Classification


1. Segmentation:
2. Feature Extraction:
3. Machine learning:

https://ccrma.stanford.edu/wiki/MIR_workshop_2014

# Retrieve Audio

1. Time Domain Representation (Wave plot)
2. Frequency Domain Representation (Spectogram)

In [ ]:
sampling_rate = 44100

song_path = path_to_small_fma + "/Hip-Hop/5.mp3"
song_path1 = path_to_small_fma + "/Electronic/100538.mp3"
song_path2 = path_to_small_fma + "/Pop/10396.mp3"

song_amp  = librosa.load(song_path, sampling_rate)[0]
song_amp1  = librosa.load(song_path1, sampling_rate)[0]
song_amp2  = librosa.load(song_path2, sampling_rate)[0]

demo_songs =[song_amp,song_amp1,song_amp2]


# Play it back!
Audio(data=song_amp, rate=sampling_rate)

In [ ]:
waveplot(demo_songs[0],sampling_rate)
plt.ylabel('Amplitude')

#### The zero crossing rate is the number of times the signal changes sign in a given period of time (usually one second).

#### The zero crossing detector looks at the second derivative computed in discrete form calclate changes 

In [ ]:
from librosa.feature import (zero_crossing_rate,spectral_centroid,
spectral_bandwidth)
from librosa import logamplitude

sampling_rate = 44100

man_features_list = []
zcr = zero_crossing_rate(demo_songs[0],sampling_rate)
plt.plot(zcr[0])
plt.xlabel('Zero Crossing Rate')
avg_zcr = np.mean(zcr)
man_features_list.append(avg_zcr)
man_features_list

# Spectral features

moments in stats i.e mean and variance, here we have anothers such as spectral centroid, bandwidth, skewness, kurtosis


Spectral features are often used to analyse harmony or timbre. Usually the product of a spectogram and a filter bank

## Pitch vs Pitch class
Chroma measures the amount of energy in each pitch class,
frame1, frame2,frame3 and collaspe down (many-to-one mapping) expectation 
of pitch class''

In [ ]:
S = librosa.feature.melspectrogram(demo_songs[0], sr=sampling_rate, n_fft=1024)
logS = librosa.logamplitude(S)
specshow(logS, sr=sampling_rate, x_axis='time', y_axis='mel')


# Constant-Q-transform

#### constant q transform is for a direct log-frequency analysis in addition, one vertical move represents one semi-tone, so things are shit invariant vertically aswell as horizontally

In [ ]:
cqt = librosa.cqt(demo_songs[0],sampling_rate)
chroma_cqt = librosa.feature.chroma_cqt(y = demo_songs[0],sr = sampling_rate)

specshow(chroma_cqt)
plt.title('chroma_cqt')
plt.colorbar()

specshow(librosa.logamplitude(cqt**2), x_axis = 'time', y_axis = 'cqt_hz')

In [ ]:
fmin = librosa.midi_to_hz(36)
C = librosa.cqt(demo_songs[0], sr=sampling_rate, fmin=fmin, n_bins=60)
logC = librosa.logamplitude(C)
librosa.display.specshow(logC, sr=sampling_rate, x_axis='time', y_axis='cqt_note', fmin=fmin, cmap='coolwarm')

In [ ]:
def extract_features(signal,sampling_rate,n_mfcc,genre):
    from librosa.feature import (zero_crossing_rate,spectral_centroid,
    spectral_bandwidth, tonnetz)

    zcr = zero_crossing_rate(signal)[0]
    norm_zcr = StandardScaler().fit_transform(zcr.reshape(1,-1))
    avg_zcr = np.mean(zcr)
    std_zcr = np.std(zcr)
    
    act_mfcc = mfcc(signal, sr=sampling_rate, n_mfcc=n_mfcc)
    norm_mfcc = StandardScaler().fit_transform(act_mfcc)
    avg_mfcc = np.mean(act_mfcc)
    std_mfcc = np.std(act_mfcc)
    
#    act_tonnetz = tonnetz(signal, sr = sampling_rate)
#    norm_tonnetz = StandardScaler().fit_transform(act_tonnetz)
#    avg_tonnetz = np.mean(act_mfcc)
#    std_tonnetz = np.std(act_mfcc)
    
    return [
        avg_zcr,std_zcr,avg_mfcc,std_mfcc,le.transform([genre])[0]
        #,avg_tonnetz,std_tonnetz
    ]

In [ ]:
n_mfcc = 12
d={}
#'Average_tonnetz','Std_tonnetz'

for i in range(len(demo_songs)):
    song_num = "song"+ str(i)
    if i ==0:
        genre = "Hip-Hop"
    elif i==1:
        genre = "Electronic"
    else:
        genre = "Pop"
        
    d[song_num] = extract_features(demo_songs[i],sampling_rate,n_mfcc,genre)


index = ['Average_zcr','Std_zcr','Average_mfcc','Std_mfcc','Genre']

In [ ]:
#Label Encoding Mapping
le_df= pd.DataFrame(data= {'Genre': complete_genre_list,
                   'Encoded':le.transform(complete_genre_list)})
le_df.to_csv('Encoder_mapping.csv')

In [ ]:
demo_df = pd.DataFrame(data=d,index =index)


In [ ]:
demo_df


In [ ]:
hdf = HDFStore('storage.h5')
hdf.put('demoFile', demo_df, format='table', data_columns=True)

In [ ]:
%%time

d={}
song_num = 0
for genre_path_name,genre_paths in paths_dict.items(): 
    song_num=song_num+1
    #for i in range(len(all_songs_path)):
    try:
        for song_path in genre_paths:           
            song_signal = librosa.load(song_path,sr=sampling_rate,mono=True)[0]
            curr_song_genre= genre_path_name[:-6]
            d[song_num]= extract_features(song_signal,sampling_rate,n_mfcc,curr_song_genre)
    except IOError as exc:
        print("Unable to locate folder")
            
complete_df = pd.DataFrame(data=d,index =index)

In [ ]:
hdf = HDFStore('storage.h5')
hdf.put('completeFile', complete_df, format='table', data_columns=True)